# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [47]:
import plotly.plotly as py
import cufflinks as cf
from ipywidgets import interact
cf.go_offline()
%matplotlib inline

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


%matplotlib inline
# import numpy and pandas



# Challenge 1 - Analysis of Variance

In this part of the lesson, we will perform an analysis of variance to determine whether the factors in our model create a significant difference in the group means. We will be examining a dataset of FIFA players. We'll start by loading the data using the code in the cell below.

In [32]:
# Run this code:

fifa = pd.read_csv('fifa.csv')

Let's examine the dataset by looking at the `head`.

In [33]:
# Your code here:
fifa.head()
fifa.tail()

,Name,Age,Nationality,Overall,Potential,Club,Value,Preferred Foot,Position,Weak Foot,Acceleration,SprintSpeed,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties
0,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,Left,RF,4.0,91.0,86.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,Right,ST,4.0,89.0,91.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,Right,LW,5.0,94.0,90.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0
3,De Gea,27,Spain,91,93,Manchester United,€72M,Right,GK,3.0,57.0,58.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,Right,RCM,5.0,78.0,76.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0


,Name,Age,Nationality,Overall,Potential,Club,Value,Preferred Foot,Position,Weak Foot,Acceleration,SprintSpeed,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties
18202,J. Lundstram,19,England,47,65,Crewe Alexandra,€60K,Right,CM,2.0,54.0,57.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0
18203,N. Christoffersson,19,Sweden,47,63,Trelleborgs FF,€60K,Right,ST,2.0,41.0,39.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0
18204,B. Worman,16,England,47,67,Cambridge United,€60K,Right,ST,3.0,70.0,69.0,55.0,32.0,45.0,32.0,15.0,48.0,43.0,55.0
18205,D. Walker-Rice,17,England,47,66,Tranmere Rovers,€60K,Right,RW,3.0,61.0,60.0,40.0,48.0,34.0,33.0,22.0,44.0,47.0,50.0
18206,G. Nugent,16,England,46,66,Tranmere Rovers,€60K,Right,CM,3.0,57.0,55.0,47.0,60.0,32.0,56.0,42.0,34.0,49.0,33.0


In [34]:
fifa = fifa.rename(columns={'Preferred Foot': 'Preferred_Foot'})

Player's values are expressed in millions of euros. We would like this column to be numeric. Therefore, let's create a numeric value column. Do this by stripping all non-numeric characters from each cell. Assign this new data to `ValueNumeric`. There is no need to multiply the value to be expressed in millions. 

In [35]:
# Your code here:

def clean(series):
    pattern='€|M|K'
    series = series.replace(regex=pattern, value='')
    return series

fifa['ValueNumeric'] = pd.to_numeric(clean(fifa['Value']))

fifa['ValueNumeric'].head()


0    110.5
1     77.0
2    118.5
3     72.0
4    102.0
Name: ValueNumeric, dtype: float64

#### We'd like to determine whether a player's preffered foot and position have an impact on their value. 

Using the `statsmodels` library, we are able to produce an ANOVA table without munging our data. Create an ANOVA table with value as a function of position and preferred foot. Recall that pivoting is performed by the `C` function.

Hint: For columns that have a space in their name, it is best to refer to the column using the dataframe (For example: for column `A`, we will use `df['A']`).

In [23]:
# Your code here:

import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('ValueNumeric ~ C(Position)', data=fifa).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

model = ols('ValueNumeric ~ C(Preferred_Foot)', data=fifa).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table


,sum_sq,df,F,PR(>F)
C(Position),9.006867e+06,26.0,4.141509,7.234254e-12
Residual,1.515654e+09,18120.0,NaN,NaN


,sum_sq,df,F,PR(>F)
C(Preferred_Foot),3.161893e+05,1.0,3.764163,0.052378
Residual,1.525186e+09,18157.0,NaN,NaN


What is your conclusion from this ANOVA?

In [24]:
# Your conclusions here:

# p-value < 0.05 rebuig , les mitjanes són diferents per Position

# p-value > 0.05 acceptació , les mitjanes són iguals per Preferred_Foot

After looking at a model of both preffered foot and position, we decide to create an ANOVA table for nationality. Create an ANOVA table for numeric value as a function of nationality.

In [25]:
# Your code here:

model = ols('ValueNumeric ~ C(Nationality)', data=fifa).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(Nationality),4.292914e+07,163.0,3.202987,1.976253e-38
Residual,1.483605e+09,18043.0,NaN,NaN


What is your conclusion from this ANOVA?

In [26]:
# p-value < 0.05 rebuig , les mitjanes són diferents

# Challenge 2 - Linear Regression

Our goal with using linear regression is to create a mathematical model that will enable us to predict the outcome of one variable using one or more additional independent variables.

We'll start by ensuring there are no missing values. Examine all variables for all missing values. If there are missing values in a row, remove the entire row.

In [53]:
# Your code here:

fifa.info()

fifa.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17918 entries, 0 to 17917
Data columns (total 21 columns):
Name              17918 non-null object
Age               17918 non-null int64
Nationality       17918 non-null object
Overall           17918 non-null int64
Potential         17918 non-null int64
Club              17918 non-null object
Value             17918 non-null object
Preferred_Foot    17918 non-null object
Position          17918 non-null object
Weak Foot         17918 non-null float64
Acceleration      17918 non-null float64
SprintSpeed       17918 non-null float64
Stamina           17918 non-null float64
Strength          17918 non-null float64
LongShots         17918 non-null float64
Aggression        17918 non-null float64
Interceptions     17918 non-null float64
Positioning       17918 non-null float64
Vision            17918 non-null float64
Penalties         17918 non-null float64
ValueNumeric      17918 non-null float64
dtypes: float64(12), int64(3), object(6)
m

,Age,Overall,Potential,Weak Foot,Acceleration,SprintSpeed,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,ValueNumeric
count,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000
mean,25.105257,66.236801,71.329334,2.947260,64.601518,64.720672,63.205994,65.323697,47.130316,55.879060,46.690870,49.995758,53.448934,48.544480,268.352232
std,4.675372,6.929243,6.144098,0.660106,14.947197,14.666098,15.892897,12.552242,19.251517,17.354347,20.691841,19.521104,14.119193,15.691563,290.149881
min,16.000000,46.000000,48.000000,1.000000,12.000000,12.000000,12.000000,17.000000,3.000000,11.000000,3.000000,2.000000,10.000000,5.000000,0.000000
25%,21.000000,62.000000,67.000000,3.000000,57.000000,57.000000,56.000000,58.000000,33.000000,44.000000,26.000000,39.000000,44.000000,39.000000,4.900000
50%,25.000000,66.000000,71.000000,3.000000,67.000000,67.000000,66.000000,67.000000,51.000000,59.000000,52.000000,55.000000,55.000000,49.000000,160.000000
75%,28.000000,71.000000,75.000000,3.000000,75.000000,75.000000,74.000000,74.000000,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,475.000000
max,45.000000,94.000000,95.000000,5.000000,97.000000,96.000000,96.000000,97.000000,94.000000,95.000000,92.000000,95.000000,94.000000,92.000000,975.000000


In [38]:
fifa = fifa.dropna(how='any',axis=0)
fifa = fifa.reset_index(drop=True)
fifa.tail()

,Name,Age,Nationality,Overall,Potential,Club,Value,Preferred_Foot,Position,Weak Foot,Acceleration,SprintSpeed,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,ValueNumeric
17913,J. Lundstram,19,England,47,65,Crewe Alexandra,€60K,Right,CM,2.0,54.0,57.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,60.0
17914,N. Christoffersson,19,Sweden,47,63,Trelleborgs FF,€60K,Right,ST,2.0,41.0,39.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,60.0
17915,B. Worman,16,England,47,67,Cambridge United,€60K,Right,ST,3.0,70.0,69.0,55.0,32.0,45.0,32.0,15.0,48.0,43.0,55.0,60.0
17916,D. Walker-Rice,17,England,47,66,Tranmere Rovers,€60K,Right,RW,3.0,61.0,60.0,40.0,48.0,34.0,33.0,22.0,44.0,47.0,50.0,60.0
17917,G. Nugent,16,England,46,66,Tranmere Rovers,€60K,Right,CM,3.0,57.0,55.0,47.0,60.0,32.0,56.0,42.0,34.0,49.0,33.0,60.0


Using the FIFA dataset, in the cell below, create a linear model predicting value using stamina and sprint speed. create the model using `statsmodels`. Print the model summary.

Hint: remember to add an intercept to the model using the `add_constant` function.

In [44]:
# Your code here:
from scipy.stats import linregress

slope, intercept, r_value, p_value, std_err = linregress(fifa.Stamina, fifa.SprintSpeed)

slope, intercept

r_value, p_value, std_err

fifa.Stamina.corr(fifa.SprintSpeed)

(0.571607343479714, 28.591661643355124)

(0.6194215111682528, 0.0, 0.0054124395622735366)

0.6194215111682528

Report your findings from the model summary. In particular, report about the model as a whole using the F-test and how much variation is predicted by the model using the r squared.

In [43]:
# Your conclusions here:

import statsmodels.api as sm

X = sm.add_constant(fifa.Stamina) # We must add the intercept using the add_constant function
Y = fifa.SprintSpeed

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

# les conclusions no són massa significatives, ja que la correlació (r) és menor a 0.8

                            OLS Regression Results                            
Dep. Variable:            SprintSpeed   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                 1.115e+04
Date:                Sun, 01 Sep 2019   Prob (F-statistic):               0.00
Time:                        11:19:49   Log-Likelihood:                -69207.
No. Observations:               17918   AIC:                         1.384e+05
Df Residuals:                   17916   BIC:                         1.384e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.5917      0.353     81.054      0.0

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Next, create a second regression model predicting value using potential. Create the model using `statsmodels` and print the model summary. Remember to add a constant term.

In [45]:
# Your code here:

slope, intercept, r_value, p_value, std_err = linregress(fifa.Potential, fifa.SprintSpeed)

slope, intercept

r_value, p_value, std_err

X = sm.add_constant(fifa.Potential) # We must add the intercept using the add_constant function
Y = fifa.SprintSpeed

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)


(0.5679521749019344, 24.209021779970463)

(0.2379333344410761, 4.714867059830263e-229, 0.01732132300668446)

                            OLS Regression Results                            
Dep. Variable:            SprintSpeed   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     1075.
Date:                Sun, 01 Sep 2019   Prob (F-statistic):          4.71e-229
Time:                        11:26:24   Log-Likelihood:                -73021.
No. Observations:               17918   AIC:                         1.460e+05
Df Residuals:                   17916   BIC:                         1.461e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         24.2090      1.240     19.522      0.0

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Report your findings from the model summary. In particular, report about the model as a whole using the F-test and how much variation is predicted by the model using the r squared.

In [13]:
# Your conclusions here:

# les conclusions encara són menys significatives, amb una correlació de només 0.21

Plot a scatter plot of value vs. potential. Do you see a linear relationship?

In [52]:
# Your code here:

fifa.iplot(y='ValueNumeric', x='Potential', categories='Nationality',
           yTitle='Value', xTitle='Potential',
           title='Potential - Value')